In [ ]:
import os
import pyart
from matplotlib import pyplot as plt
import numpy as np
import time
import datetime
%matplotlib inline

In [ ]:
data_dir = '/eagle/projects/CPOL/SAIL/202111/'
all_files = os.listdir(data_dir)
all_files.sort()

In [ ]:
base_scan_ppi = '1_PPI.nc'
ppi_pattern = 'PPI.nc'
base_scans = []
volumes = []
ppis = []
in_volume = False
for file in all_files:
    if ppi_pattern in file:
        ppis.append(file)
    if base_scan_ppi in file:
        base_scans.append(file)
    


In [ ]:
n_tilts = 8

In [ ]:
index = np.where(np.array(ppis) == base_scans[0])[0][0]
print(index)
print(ppis[index: index+n_tilts])

In [ ]:
volumes = []
for base in base_scans:
    base_scan_index = np.where(np.array(ppis) == base)[0][0]
    #print(base_scan_index)
    volume = ppis[base_scan_index: base_scan_index+n_tilts]
    volumes.append(volume)

In [ ]:
volumes[0]

In [ ]:
base_radar = pyart.io.read(data_dir+volumes[0][0])
radars = [pyart.io.read(data_dir+sw) for sw in volumes[0][1::]]

In [ ]:
base_radar.elevation['data'].mean()

In [ ]:
radars

In [ ]:
for rad in radars:
    print(rad.elevation['data'].mean())
    print(rad.fixed_angle['data'])

In [ ]:
for rad in radars:
    base_radar = pyart.util.join_radar(base_radar, rad)

In [ ]:
base_radar.nsweeps

In [ ]:
base_radar.info()

In [ ]:
base_radar.fixed_angle

In [ ]:
def radar_glue(b_radar, radar_list):
    for rad in radar_list:
        b_radar = pyart.util.join_radar(b_radar, rad)
    
    return b_radar

In [ ]:
def volume_from_list(vlist, base_dir):
    base_radar = pyart.io.read(base_dir+vlist[0])
    radars = [pyart.io.read(base_dir+sw) for sw in vlist[1::]]
    return radar_glue(base_radar, radars)

In [ ]:
fourth_volume = volume_from_list(volumes[400], data_dir)

In [ ]:
figme = plt.figure(figsize=[15,10])
display = pyart.graph.RadarMapDisplay(fourth_volume)
    
# Add our reflectivity (DBZ) field to the plot, including our gatefilter
display.plot_ppi_map('DBZ', 0, vmin=-20, vmax=40.)


In [ ]:
figme = plt.figure(figsize=[15,10])
display = pyart.graph.RadarMapDisplay(fourth_volume)
    
# Add our reflectivity (DBZ) field to the plot, including our gatefilter
display.plot_ppi_map('DBZ', 7, vmin=-20, vmax=40.)


In [ ]:
for vvl in volumes[0:1]:
    if len(vvl) == n_tilts:
        out_radar = volume_from_list(vvl, data_dir)
        print(out_radar.time['units'][14:])
        ff = time.strptime(out_radar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
        dt = datetime.datetime.fromtimestamp(time.mktime(ff)) + datetime.timedelta(seconds= int(out_radar.time['data'][0]))
        strform = dt.strftime('/eagle/projects/CPOL/SAIL/202111_glued/xprecipradar_guc_volume_%Y%m%d-%H%M%S.b1.nc')
        print(strform)
        #FIX for join issue.. to be fixed in Py-ART
        out_radar.sweep_mode['data']=np.tile(radars[0].sweep_mode['data'], n_tilts)
        pyart.io.write_cfradial(strform, out_radar)
        

In [ ]:
acompletelynewradar = pyart.io.read('/eagle/projects/CPOL/SAIL/202111_glued/xprecipradar_guc_volume_20211110-000110.b1.nc')

In [ ]:
figme = plt.figure(figsize=[15,10])
display = pyart.graph.RadarMapDisplay(acompletelynewradar)
    
# Add our reflectivity (DBZ) field to the plot, including our gatefilter
display.plot_ppi_map('DBZ', 7, vmin=-20, vmax=40.)


In [ ]:
def granule(Dvolume):
    data_dir = '/eagle/projects/CPOL/SAIL/202111/'
    if len(Dvolume) == 8:
        base_rad = pyart.io.read(data_dir+Dvolume[0])
        out_radar = volume_from_list(Dvolume, data_dir)
        print(out_radar.time['units'][14:])
        ff = time.strptime(out_radar.time['units'][14:], '%Y-%m-%dT%H:%M:%SZ')
        dt = datetime.datetime.fromtimestamp(time.mktime(ff)) + datetime.timedelta(seconds= int(out_radar.time['data'][0]))
        strform = dt.strftime('/eagle/projects/CPOL/SAIL/202111_glued/xprecipradar_guc_volume_%Y%m%d-%H%M%S.b1.nc')
        print(strform)
        #FIX for join issue.. to be fixed in Py-ART
        out_radar.sweep_mode['data']=np.tile(base_rad.sweep_mode['data'], n_tilts)
        pyart.io.write_cfradial(strform, out_radar)

In [ ]:
granule(volumes[0])

In [ ]:
from dask.distributed import Client, LocalCluster

cluster = LocalCluster()

cluster.scale(16)  # Sets the number of workers to 10
cluster.adapt(minimum=8, maximum=16)
client = Client(cluster)
client

In [ ]:
future = client.map(granule, volumes)

In [ ]:
my_data = client.gather(future)

In [ ]:
client